In [4]:
from data_pipeline import *
import time
import numpy as np
from ghcn_helper import *


## Check correctness

In [5]:
ghcn = GHCN("20180101", "20181231", ["TMAX", "TMIN"], graph_freezing=150)
yearbook = build_yearbook("20180101", "20181231", ["TMAX", "TMIN"])
df = yearbook[2018]
df


latitude  longitude  TMAX  TMIN
date     station_id                                  
20180101 AE000041196   25.3330    55.5170  25.9  11.2
         AJ000037575   41.5500    46.6670   7.4   5.3
         AJ000037675   41.3670    48.5170   4.5   0.3
         AJ000037735   40.7167    46.4167  14.3   5.4
         AJ000037756   40.5330    48.9330  10.0   0.6
...                        ...        ...   ...   ...
20181231 WA007401540  -22.5670    17.1000  29.8  15.4
         WA007848390  -22.4830    17.4670  28.2  15.2
         WA012084750  -17.9170    19.7670  25.0  19.0
         WF000917530  -13.2330  -176.1670  28.8  25.8
         ZI000067775  -17.9170    31.1330  28.5  16.6

[4234648 rows x 4 columns]

In [6]:
df_day_one = df.loc["20180101"]
df_day_one = df_day_one[["TMAX", "TMIN"]]
df_day_one


,TMAX,TMIN
station_id,,
AE000041196,25.9,11.2
AJ000037575,7.4,5.3
AJ000037675,4.5,0.3
AJ000037735,14.3,5.4
AJ000037756,10.0,0.6
...,...,...
VQW00011640,30.0,23.9
WA007848390,34.1,11.6
WA007878380,37.0,14.8


In [7]:
msk_lap, x, non_zero_idx = ghcn[0]
blacklist = ghcn.blacklist
df_day_one = df_day_one.drop(blacklist, axis=0, errors="ignore")

non_zero_x = x[non_zero_idx]
print(non_zero_x)

assert np.allclose(df_day_one.values, non_zero_x.numpy())


---previous Laplacian expired or no previous Laplacian; building new---
Found 13778 stations in total in date range 20180101-20180530
Successfully converged after 111 steps
Difference between ML @ v and lambda * v:  6.754723049376121
tensor([[25.9000, 11.2000],
        [ 7.4000,  5.3000],
        [ 4.5000,  0.3000],
        ...,
        [37.0000, 14.8000],
        [26.7000, 12.2000],
        [30.9000, 24.3000]], dtype=torch.float64)


## Showcase dataset usage

In [8]:
ghcn = GHCN("20180101", "20181231", ["TMAX", "TMIN"], graph_freezing=150)


In [9]:
for i in range(365):
    print("Iter: ", i)
    t = time.time()
    msk_lap, x, non_zero_idx = ghcn[i]
    print("Time Taken: ", time.time() - t)
    


Iter:  0
---previous Laplacian expired or no previous Laplacian; building new---
Found 13778 stations in total in date range 20180101-20180530
Successfully converged after 94 steps
Difference between ML @ v and lambda * v:  7.203679244569644
Time Taken:  20.119462251663208
Iter:  1
Time Taken:  0.20965290069580078
Iter:  2
Time Taken:  0.21915984153747559
Iter:  3
Time Taken:  0.21024584770202637
Iter:  4
Time Taken:  0.26366400718688965
Iter:  5
Time Taken:  0.20982599258422852
Iter:  6
Time Taken:  0.21443915367126465
Iter:  7
Time Taken:  0.26282525062561035
Iter:  8
Time Taken:  0.21117019653320312
Iter:  9
Time Taken:  0.22769808769226074
Iter:  10
Time Taken:  0.21407103538513184
Iter:  11
Time Taken:  0.26043272018432617
Iter:  12
Time Taken:  0.20634818077087402
Iter:  13
Time Taken:  0.2639739513397217
Iter:  14
Time Taken:  0.3641479015350342
Iter:  15
Time Taken:  0.25267910957336426
Iter:  16
Time Taken:  0.22318387031555176
Iter:  17
Time Taken:  0.217634916305542
Iter:  1

Time Taken:  0.23329806327819824
Iter:  181
Time Taken:  0.23136472702026367
Iter:  182
Time Taken:  0.294586181640625
Iter:  183
Time Taken:  0.294191837310791
Iter:  184
Time Taken:  0.24102020263671875
Iter:  185
Time Taken:  0.24420619010925293
Iter:  186
Time Taken:  0.28075695037841797
Iter:  187
Time Taken:  0.2432398796081543
Iter:  188
Time Taken:  0.22885799407958984
Iter:  189
Time Taken:  0.29529523849487305
Iter:  190
Time Taken:  0.27103304862976074
Iter:  191
Time Taken:  0.2692749500274658
Iter:  192
Time Taken:  0.22734522819519043
Iter:  193
Time Taken:  0.29003095626831055
Iter:  194
Time Taken:  0.254741907119751
Iter:  195
Time Taken:  0.26746487617492676
Iter:  196
Time Taken:  0.3333001136779785
Iter:  197
Time Taken:  0.23203802108764648
Iter:  198
Time Taken:  0.23423409461975098
Iter:  199
Time Taken:  0.22362089157104492
Iter:  200
Time Taken:  0.2795217037200928
Iter:  201
Time Taken:  0.22556805610656738
Iter:  202
Time Taken:  0.228010892868042
Iter:  203


Time Taken:  0.2774620056152344
Iter:  363
Time Taken:  0.2336130142211914
Iter:  364
Time Taken:  0.2155132293701172


In [10]:
ghcn = GHCN("20180101", "20181231", ["TMAX", "TMIN"], graph_freezing=3)
sampler = Sampler(times=32, size=0.01)
ghcn_impainter = GHCNImpainter(ghcn, sampler)


In [11]:
for i in range(16):
    print("ITER: ", i)
    t = time.time()
    further_masked_laplacian, masked_x, idx_to_mask, masked_value = ghcn_impainter[i]
    print("TIME TAKEN: ", time.time() - t)


ITER:  0
---previous Laplacian expired or no previous Laplacian; building new---
Found 12533 stations in total in date range 20180101-20180103
Successfully converged after 103 steps
Difference between ML @ v and lambda * v:  5.446629300112339
TIME TAKEN:  14.95556092262268
ITER:  1
TIME TAKEN:  0.14760184288024902
ITER:  2
TIME TAKEN:  0.15729498863220215
ITER:  3
TIME TAKEN:  0.20305895805358887
ITER:  4
TIME TAKEN:  0.15541315078735352
ITER:  5
TIME TAKEN:  0.1477031707763672
ITER:  6
TIME TAKEN:  0.16678500175476074
ITER:  7
TIME TAKEN:  0.20081806182861328
ITER:  8
TIME TAKEN:  0.14926695823669434
ITER:  9
TIME TAKEN:  0.1437990665435791
ITER:  10
TIME TAKEN:  0.15725183486938477
ITER:  11
TIME TAKEN:  0.2033383846282959
ITER:  12
TIME TAKEN:  0.15298891067504883
ITER:  13
TIME TAKEN:  0.1491680145263672
ITER:  14
TIME TAKEN:  0.1535649299621582
ITER:  15
TIME TAKEN:  0.20044207572937012
